# Facility Location for temporal markets using PuLP and UrbanPy

### General imports

In [ ]:
import sys
sys.path.append('..')
import urbanpy as up

import pandas as pd
import geopandas as gpd
import shapely
import numpy as np
import re
from pulp import *

### Data preparation

First, we need our set of possible facilities to activate. To construct it we will

* Download a polygon from a Lima district
* Get its bounds
* Download data from possible parks and pitches

In [ ]:
sjl = up.download.nominatim_osm('San Juan de Lurigancho, Lima')
sjl.crs = 'EPSG:4326'

Now we are going to create a custom query for the overpass api to download parks an pitches within the district polygon. You can try your own cutom querys [here](https://overpass-turbo.eu/).

In [ ]:
query = """
[timeout:120][out:json][bbox];
(
  way["leisure"~"park|pitch"];
);
out body geom;
"""
response = up.download.overpass_pois(sjl.total_bounds, custom_query=query)

Now, we are going to create a GeoDataFrame using the query results

In [ ]:
data = response.json() # get json data
parks_pitchs_df = pd.DataFrame.from_dict(data['elements']) # transform data to dataframe

In [ ]:
# shell_from_geometry is a helper function to create the input for a Polygon using the overpass geometry
parks_pitchs_df['shell'] = parks_pitchs_df['geometry'].apply(up.utils.shell_from_geometry)

In [ ]:
# Use shell to instanciate a Polygon for each row
parks_pitchs_geometry = parks_pitchs_df['shell'].apply(shapely.geometry.Polygon)

In [ ]:
# Create a GeoDataFrame using the DataFrame and the calculated geometry
parks_pitchs_gdf = gpd.GeoDataFrame(parks_pitchs_df, geometry=parks_pitchs_geometry)

We need to process the polygon data, getting the centroid is a good approximation

In [ ]:
parks_pitchs_gdf['lat'] = parks_pitchs_gdf.geometry.centroid.y
parks_pitchs_gdf['lon'] = parks_pitchs_gdf.geometry.centroid.x

Construct the candidate set from Overpass POIs

In [ ]:
candidates = parks_pitchs_gdf[['id', 'lat', 'lon']]
candidates = gpd.GeoDataFrame(candidates, geometry=gpd.points_from_xy(candidates['lon'], candidates['lat']))
candidates.crs = 'EPSG:4326'

In [ ]:
candidates.head()

### Concatenating candidates with the national market census

In [ ]:
markets = pd.read_csv('input/market_db.csv')
markets = gpd.GeoDataFrame(markets, geometry=gpd.points_from_xy(markets['longitude'], markets['latitude']))
markets.crs = 'EPSG:4326'

In [ ]:
merc = markets[markets.within(sjl.geometry[0])]

In [ ]:
merc = merc[['longitude', 'latitude']].reset_index()

In [ ]:
merc = gpd.GeoDataFrame(merc, geometry=gpd.points_from_xy(merc['longitude'], merc['latitude']))

In [ ]:
merc = merc.rename(columns={'index': 'id', 'longitude': 'lon', 'latitude': 'lat'})

In [ ]:
merc.shape

In [ ]:
markets = markets.loc[merc.id]

In [ ]:
markets['aforo'] = markets.apply(
    lambda row: row['Area construida']*2 if row['Tipo de mercado']=='Minorista' else row['Area construida']*5,
    axis=1
)

In [ ]:
markets.isna().sum()

In [ ]:
candidates = candidates[['id', 'lat', 'lon', 'geometry']]

In [ ]:
candidates = gpd.GeoDataFrame(pd.concat([candidates, merc])).reset_index(drop=True)

In [ ]:
candidates.head()

### Creating the demand an "clients" for the FLP

Now, we need to estimate the total population moving to these markets. To achieve this we need to

* Download HDX data
* Filter it to our district
* Convert it to hexagons

In [ ]:
pop = up.download.hdx_dataset("4e74db39-87f1-4383-9255-eaf8ebceb0c9/resource/317f1c39-8417-4bde-a076-99bd37feefce/download/population_per_2018-10-01.csv.zip")

In [ ]:
pop_sjl = up.geom.filter_population(pop, sjl)

In [ ]:
hex_sjl = up.geom.gen_hexagons(7, sjl)

In [ ]:
hex_sjl.plot()

Merging both layers

In [ ]:
pop_sjl.isna().sum()

In [ ]:
hex_sjl.isna().sum()

In [ ]:
hex_sjl = up.geom.merge_shape_hex(
    hex_sjl, 
    pop_sjl, 
    how='inner', 
    op='intersects', 
    agg={'population_2020': 'sum'}
)

In [ ]:
hex_sjl.isna().sum()

In [ ]:
hex_sjl.fillna(0, inplace=True)

In [ ]:
hex_sjl.plot(column='population_2020', legend=True)

## Distance matrix calculation

To estimate the cost for our FLP, we will use walking travel time. For this we need to

* Setup the OSRM server
* Get the distance matrix
* Shutdown the server

In [ ]:
up.routing.start_osrm_server('peru')

To compute_osrm_dist_matrix we need Point geometry (We are going to use each hexagon centroid)

In [ ]:
hex_sjl_centroids = hex_sjl.copy() # Copy original gdf
hex_sjl_centroids.geometry = hex_sjl_centroids.geometry.centroid # Replace Polygon for Point/Polygon Centroid

In [ ]:
dmat = up.routing.compute_osrm_dist_matrix(hex_sjl_centroids, candidates, 'walking')

In [ ]:
up.routing.stop_osrm_server()

In [ ]:
distance, duration = dmat

In [ ]:
cost_mat = duration.T

### Constructing PuLP sets and variables

Set an the number of facilities to be activated

In [ ]:
p = merc.shape[0]+12

In [ ]:
p

Build the facility and customer set as lists, as per PuLP requirements

In [ ]:
customers = list(hex_sjl_centroids.index)
facilities = [f'FAC_{i}' for i in candidates.index]

Now we create dictionaries for the demand and cost, associating each customer (hexagon) to the demand (population) and each facility (park/pitch) the respective cost (travel time) to each customer 

In [ ]:
demand = {i: hex_sjl.loc[i, 'population_2020'] for i in hex_sjl.index}

In [ ]:
cost_dict = {facilities[i]: {customers[j]: cost_mat[i][j] for j in hex_sjl.index} for i in candidates.index}

In [ ]:
merc.id

In [ ]:
capacity = {facilities[i]: markets.loc[i, 'aforo'] for i in merc.id}

Create problem variable for PuLP

In [ ]:
prob = LpProblem('FLP_Markets_SJL', LpMinimize)

We create the decision variable $x_{ij}$, representing the percentage of service assigned from a facility to a customer, setting 0 as the lower bound

In [ ]:
x = LpVariable.dicts('Service', 
                    [(i,j) for j in customers for i in facilities],
                    0)

Now we create the decision variable to activate a facility $y_i$

In [ ]:
y = LpVariable.dicts('Activation',
                     facilities,
                     0,1, LpBinary)

Setting the objective function $$ \sum_{i=1}^{n} \sum_{j=1}^{m} d_{j} c_{ij} x_{ij} $$

In [ ]:
prob += lpSum(lpSum(demand[j]*cost_dict[i][j]*x[i,j] for i in facilities) for j in customers)

We add the first constraint $$ \sum_{i=1}^{n} x_{ij} = 1 \quad \forall j \in \text{Customers}$$

In [ ]:
for j in customers:
    prob += lpSum(x[i,j] for i in facilities) == 1

Adding the second constraint $$ \sum_{i=1}^{n} y_{i} = p $$

In [ ]:
prob += lpSum(y[i] for i in facilities) == p

Third constraint $$ x_{i,j} ≤ y_{i} \quad \forall i \in \text{Facilities} \quad \forall j \in \text{Customers}$$

In [ ]:
for i in facilities:
    for j in customers:
        prob += x[i,j] <= y[i]

We need to keep the markets active, focusing on activating only additional facilities (avoid selecting already active markets)

In [ ]:
for i in facilities[-merc.shape[0]:]:
    prob += y[i] == 1

Finally, as to maintain proper social distancing, we set a capacity constraint $$ \sum_{j=1}^{m} x_{ij} ≤ c_{i} \quad \forall i \in \text{Facilities}$$

In [ ]:
for i in capacity:
    prob += lpSum(x[i,j] for j in customers) <= capacity[i]

Write the problem to a .lp file (optional)

In [ ]:
prob.writeLP("FLP_Markets_SJL.lp");

Solve (a custom solver may be added in the solve method)

In [ ]:
prob.solve()

Check solution status

In [ ]:
print("Status:", LpStatus[prob.status])

Obtain the demand assignments

In [ ]:
x_vars = [[0 for j in range(len(customers))] for i in range(len(facilities))]

for v in prob.variables():
    if 'Activation' not in v.name:
        i, j = re.findall('\d+', v.name)
        x_vars[int(i)][int(j)] = v.varValue

In [ ]:
y_vars = [v.varValue for v in prob.variables() if 'Activation' in v.name]

In [ ]:
x_vars, y_vars = np.array(x_vars), np.array(y_vars)

In [ ]:
np.save('output/assignments.npy', x_vars)
np.save('output/facilities.npy', y_vars)